# Event Processing Patterns in Python
This notebook demonstrates common in-memory event-processing algorithms and data structures, using synthetic event streams. Each section includes detailed comments and example usage.

In [ ]:
# Core imports used across examples
import collections
from collections import deque, Counter, OrderedDict
import heapq
import bisect
import random
from dataclasses import dataclass
from typing import List, Tuple, Dict

# Utility for printing results
def show(title, result):
    print(f"## {title}\n", result, "\n")

## 1. Hash Map / Counter
Count occurrences, group by key, dedupe, or track last-seen values in O(1) per event.

In [ ]:
# Synthetic event stream: list of user IDs
events = [random.choice(['alice', 'bob', 'carol', 'dave']) for _ in range(20)]
show('Events', events)

# 1.a Count occurrences using Counter
counts = Counter(events)
show('Counts per user', counts)

# 1.b Track last-seen index for each user
last_seen = {}
for idx, user in enumerate(events):
    last_seen[user] = idx
show('Last-seen positions', last_seen)

## 2. Sliding Window
Maintain data for the last *k* items (e.g., moving averages or rate limits).

In [ ]:
# Synthetic timestamped readings (seconds)
readings = [(i, random.uniform(0, 100)) for i in range(30)]
window_size = 5  # seconds
dq = deque()
sum_vals = 0
moving_avg = []

for timestamp, value in readings:
    # Add new event
    dq.append((timestamp, value))
    sum_vals += value
    # Expire old events
    while dq and dq[0][0] < timestamp - window_size:
        _, old_val = dq.popleft()
        sum_vals -= old_val
    # Compute average over current window
    moving_avg.append((timestamp, sum_vals / len(dq)))

show('Moving average (window=5)', moving_avg)

## 3. Min/Max-Heap
Maintain the top-K elements or next expiry in O(log N) per update.

In [ ]:
# Synthetic scores to maintain top-3
scores = [random.randint(0, 100) for _ in range(15)]
show('Scores stream', scores)

# Min-heap of size K to keep top K
K = 3
min_heap = []
for score in scores:
    if len(min_heap) < K:
        heapq.heappush(min_heap, score)
    else:
        heapq.heappushpop(min_heap, score)
show('Top-3 scores', sorted(min_heap, reverse=True))

## 4. Ordered Map / Range Queries
Support range counts or next/prev lookups in sorted order.

In [ ]:
# Synthetic timestamps
timestamps = sorted(random.randint(0, 100) for _ in range(20))
show('Timestamps', timestamps)

# Use bisect to count events in a given range [low, high]
low, high = 20, 50
left = bisect.bisect_left(timestamps, low)
right = bisect.bisect_right(timestamps, high)
show(f'Count in [{low}, {high}]', right - left)

## 5. Interval Merge / Sweep-Line
Merge overlapping intervals or compute total covered time.

In [ ]:
# Synthetic intervals (start, end)
intervals = [(random.randint(0, 60), random.randint(60, 120)) for _ in range(10)]
# Normalize start < end
intervals = [(min(s, e), max(s, e)) for s, e in intervals]
intervals.sort()
show('Original intervals', intervals)

# Merge logic
merged = []
for start, end in intervals:
    if not merged or start > merged[-1][1]:
        merged.append((start, end))
    else:
        merged[-1] = (merged[-1][0], max(merged[-1][1], end))
show('Merged intervals', merged)

# Compute total covered time
total = sum(e - s for s, e in merged)
show('Total covered time', total)

## 6. LRU Cache
Evict least-recently-used items when capacity is exceeded, in O(1) per access.

In [ ]:
class LRUCache:
    def __init__(self, capacity: int):
        self.cache = OrderedDict()
        self.capacity = capacity

    def get(self, key):
        if key not in self.cache:
            return None
        # Move to end (most recent)
        value = self.cache.pop(key)
        self.cache[key] = value
        return value

    def put(self, key, value):
        if key in self.cache:
            self.cache.pop(key)
        elif len(self.cache) >= self.capacity:
            # Pop oldest (first key)
            self.cache.popitem(last=False)
        self.cache[key] = value

# Demo
ops = [('put', i, i*10) for i in range(5)] + [('get', 2, None), ('put', 5, 50)]
cache = LRUCache(capacity=3)
for op, key, val in ops:
    if op == 'put':
        cache.put(key, val)
    else:
        cache.get(key)
show('Final LRU state', list(cache.cache.items()))

## 7. Reservoir Sampling
Randomly sample k items from a stream of unknown size with O(k) memory.

In [ ]:
def reservoir_sample(stream, k=1):
    # Initialize reservoir
    reservoir = []
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            # Replace with decreasing probability
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir

# Demo: sample one event ID
events = list(range(100))
sample = reservoir_sample(events, k=1)
show('Reservoir sample (k=1)', sample)

## 8. Graph Traversal / Union-Find
Maintain connectivity as edges arrive and detect cycles.

In [ ]:
# Union-Find implementation
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0]*n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        rx, ry = self.find(x), self.find(y)
        if rx == ry:
            return False  # cycle
        if self.rank[rx] < self.rank[ry]:
            self.parent[rx] = ry
        elif self.rank[rx] > self.rank[ry]:
            self.parent[ry] = rx
        else:
            self.parent[ry] = rx
            self.rank[rx] += 1
        return True

# Synthetic union events
edges = [(random.randint(0, 9), random.randint(0, 9)) for _ in range(15)]
uf = UnionFind(10)
results = [(u, v, uf.union(u, v)) for u, v in edges]
show('Edge union results (cycle? False means merged)', results)

## 9. Streaming Max-Profit (DP Over Prices)
Compute max profit in one-pass as prices stream in.

In [ ]:
# Synthetic stock prices
prices = [random.randint(10, 100) for _ in range(20)]
min_price = float('inf')
max_profit = 0
for price in prices:
    min_price = min(min_price, price)  # best buy-so-far
    profit = price - min_price         # possible sell gain
    max_profit = max(max_profit, profit)

show('Prices', prices)
show('Max profit', max_profit)